In [14]:
import os
import json

In [15]:
"""
Creating the "inventory" as a list of dictionaries.
The list will look like this:
"""

inventory = [
        {
            "name": "apples",
            "quantity": 52,
            "sold_quantity": 0,
            "sale_price": 0.29,
            "cost_price": 0.14
        },
        {
            "name": "bananas",
            "quantity": 23,
            "sold_quantity": 0,
            "sale_price": 0.89,
            "cost_price": 0.37
        },
        {
            "name": "onions",
            "quantity": 66,
            "sold_quantity": 0,
            "sale_price": 1.49,
            "cost_price": 0.9
        }
    ]

In [16]:
"""
Write the content of the "inventory" dictionary to the "inventory.json" file:

First, define the path of the .json file. In this case, the path is the same of the Jupyter notebook;
Second, we define  a function that writes the content of the "inventory" variable into the file
"""
json_file_path = os.path.abspath("inventory.json")


def write_json(inventory):
    
    with open(json_file_path, "w", encoding="utf-8") as json_file:
        json.dump(inventory, json_file, indent=4, ensure_ascii=False)

In [17]:
# Find the .json file in the directory

def json_file_finder():
    return os.path.isfile(json_file_path)

"""
The json_file_finder function  checks if the file already exists and, if not, it creates the file. 
This is to ensure the file is present before reading or writing.
"""

if not json_file_finder():
    write_json(inventory)
    print(f"'inventory.json' file created.")

else:
    print(f"'inventory.json' file already exists.")

'inventory.json' file created.


In [18]:
# Validation of the user input

def get_valid_input(prompt, input_type):

    """
    The get_valid_input function is used to validate user inputs. 
    It handles different data types and provides informative error messages.
    """
        
    while True:
        try:
            input_value = input_type(input(prompt))

            if isinstance(input_value, (int, float)):
                if input_value < 0:
                    print("Value can't be negative!")
    
                else:
                    return input_value

            elif isinstance(input_value, str):
                return input_value
    
            else:
                print("Invalid format!\n")

        except ValueError:
            print("Invalid format!\n")


In [19]:
# List products with their respective quantities in stock and prices

def list_products():
    
    try:
        json_file_finder()
    
    except FileNotFoundError as e:
        print(e)
        return
    
    """
    To list the products, we open the file in read mode "r".
    The 'list_products' indeed lists the products with their quantities, 
    selling prices, and cost prices, by reading the JSON file we've created.
    """

    with open("inventory.json", "r") as json_file:
        inventory = json.load(json_file)
    
        print("Here is the list of available products:")

        for product in inventory:
            print(
                f"Product Name: {product['name']} | Quantity in stock: {product['quantity']}"
                f" | Selling Price: €{product['sale_price']:.2f} | Cost Price: €{product['cost_price']}"
            )
        print("")


In [20]:
# Display the help menu with all available commands

def help_menu():

    print("Here is the list of available commands:\n"
          "- add: add a product to the inventory.\n"
          "- list: list products in stock.\n"
          "- sale: record a sale.\n"
          "- profits: show gross and net profits.\n"
          "- help: show available commands.\n"
          "- close: exit the program.\n")

In [21]:
# Calculate gross and net profits

def show_profits():
    try:
        json_file_finder()

    except FileNotFoundError as e:
        print(e)
        return

    with open("inventory.json", "r", encoding="utf-8") as json_file:
        inventory = json.load(json_file)

    gross_profit = 0
    net_profit = 0

    for product in inventory:
        gross_profit += product["sale_price"] * product["sold_quantity"]
        net_profit += (product["sale_price"] - product["cost_price"]) * product["sold_quantity"]

    print(f"Gross profit: €{gross_profit:.2f}\n" 
          f"Net profit: €{net_profit:.2f}\n")

In [22]:
def get_remaining_quantity(product_name):
    
    with open("inventory.json", "r") as json_file:
        inventory = json.load(json_file)
        for product in inventory:
            if product["name"] == product_name:
                return product["quantity"]
    return 0  # Return 0 if the product is not found (handle this case as needed)

# Summarize the characteristics of an added product to the inventory

def sold_product_summary(name, sold_quantity, sale_price, cost_price):
    
    remaining_quantity = get_remaining_quantity(name)  # Function to get the remaining quantity
    
    print(f"Updated info of the product sold:\n"
          f"Product Name: {name}\n"
          f"Quantity in stock: {remaining_quantity}\n"
          f"Selling Price: €{sale_price:.2f}\n"
          f"Product Cost: €{cost_price:.2f}\n")

In [23]:
# Record a sale

def register_sale():
    try:
        json_file_finder()

    except FileNotFoundError as e:
        print(e)
        return

    total_revenue = 0

    while True:
        sold_product = input("Enter the name of the product to be sold: ")

        with open("inventory.json", "r") as json_file:
            inventory = json.load(json_file)

        product_found = False

        for product in inventory:
            if product["name"] == sold_product:
                sold_quantity = get_valid_input("Enter the number of units to sell: ", int)

                if product["quantity"] < sold_quantity:
                    raise ValueError(f"Not enough units of {sold_product} in stock!\n")

                else:
                    # Update the quantity correctly
                    product["quantity"] -= sold_quantity
                    product["sold_quantity"] += sold_quantity
                    product_found = True
                    product_revenue = product["sale_price"] * sold_quantity
                    total_revenue += product_revenue

                    break

        if product_found:
            write_json(inventory)
            print(f"The sale has been successfully registered.\n"
                  f"Summary:\n"
                  f"Sold product: {sold_product}\n"
                  f"Quantity sold: {sold_quantity}\n"
                  f"Sale revenues: €{product_revenue:.2f}\n")

            sold_product_summary(sold_product, sold_quantity, product["sale_price"], product["cost_price"])

            response = input("Do you want to record another sale (y/n)? ")
            if response != "y":
                break

        else:
            raise ValueError(f"The product \"{sold_product}\" is not present in stock.\n")

    print(f"TOTAL: €{total_revenue:.2f}\n")

In [24]:
# Summarize the characteristics of an added product to the inventory

def added_product_summary(name, quantity, sale_price, cost_price):
    
    print(f"The article has been successfully added.\n"
        f"Updated info:\n"
        f"Product Name: {name}\n"
        f"Quantity in stock: {quantity}\n"
        f"Selling Price: €{sale_price:.2f}\n"
        f"Product Cost: €{cost_price:.2f}\n")

In [25]:
# Add new products or increase units of existing products

def add_product():
    try:
        json_file_finder()
    except FileNotFoundError as e:
        print(e)
        return

    name = get_valid_input("Enter the product name: ", str)
    quantity = get_valid_input("Enter the quantity to add: ", int)

    for listed_product in inventory:
        if listed_product["name"] == name:
            listed_product["quantity"] += quantity
            sale_price = listed_product["sale_price"]
            cost_price = listed_product["cost_price"]

            added_product_summary(name, listed_product["quantity"], sale_price, cost_price)

            break
    else:
        sale_price = get_valid_input("Selling price: €", float)
        cost_price = get_valid_input("Cost price: €", float)

        new_product = {
            "name": name,
            "quantity": quantity,
            "sold_quantity": 0,
            "sale_price": sale_price,
            "cost_price": cost_price
        }

        inventory.append(new_product)
        added_product_summary(name, quantity, sale_price, cost_price)

    write_json(inventory)

In [26]:
cmd = None

while cmd != "close":
    cmd = input("Enter a command: ")

    try:
        if cmd == "sale":
            register_sale()

        elif cmd == "profits":
            show_profits()

        elif cmd == "add":
            add_product()

        elif cmd == "list":
            list_products()

        elif cmd == "help":
            help_menu()

        elif cmd == "close":
            print("Goodbye!")
            break

        else:
            print("Invalid command.")
            help_menu()
            #break

    except ValueError as e:
        print(e)

Here is the list of available products:
Product Name: apples | Quantity in stock: 52 | Selling Price: €0.29 | Cost Price: €0.14
Product Name: bananas | Quantity in stock: 23 | Selling Price: €0.89 | Cost Price: €0.37
Product Name: onions | Quantity in stock: 66 | Selling Price: €1.49 | Cost Price: €0.9

The article has been successfully added.
Updated info:
Product Name: kiwi
Quantity in stock: 10
Selling Price: €3.00
Product Cost: €1.80

Here is the list of available products:
Product Name: apples | Quantity in stock: 52 | Selling Price: €0.29 | Cost Price: €0.14
Product Name: bananas | Quantity in stock: 23 | Selling Price: €0.89 | Cost Price: €0.37
Product Name: onions | Quantity in stock: 66 | Selling Price: €1.49 | Cost Price: €0.9
Product Name: kiwi | Quantity in stock: 10 | Selling Price: €3.00 | Cost Price: €1.8

The sale has been successfully registered.
Summary:
Sold product: kiwi
Quantity sold: 5
Sale revenues: €15.00

Updated info of the product sold:
Product Name: kiwi
Qua